## 模型3: 多目标规划：加权求和

$$x_5 = max\{q_i x_i\}$$

In [118]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [119]:
s, r, q, p, u = gp.multidict(
    {
        "s0": [0.05, 0, 0, 0],
        "s1": [0.28, 0.025, 0.01, 103],
        "s2": [0.21, 0.015, 0.02, 198],
        "s3": [0.23, 0.055, 0.045, 52],
        "s4": [0.25, 0.026, 0.065, 40],
    }
)

In [120]:
# M：总投资 w:投资偏好系数
M = 10000
w = 0.5

In [121]:
m = gp.Model("模型3")
x = m.addVars(s, name="x")
x5 = m.addVar(name="x5")

In [122]:
m.addConstrs((q[i] * x[i] <= x5 for i in s[1:]), name="x5定义")

{'s1': <gurobi.Constr *Awaiting Model Update*>,
 's2': <gurobi.Constr *Awaiting Model Update*>,
 's3': <gurobi.Constr *Awaiting Model Update*>,
 's4': <gurobi.Constr *Awaiting Model Update*>}

In [123]:
con1_coef = {i: p[i] + 1 for i in s}
m.addConstr(x.prod(con1_coef) == M, name="总投资限制")

<gurobi.Constr *Awaiting Model Update*>

In [124]:
results = []
for w in np.arange(0, 0.96, 0.01):

    m.setObjective(w * x5 - (1 - w) * x.prod(coef), sense=GRB.MINIMIZE)
    m.optimize()
    result = [x[i].X for i in x]
    result.append(w)
    result.append(x5.X)
    results.append(result)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 6 columns and 13 nonzeros
Model fingerprint: 0x6e30dddd
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [5e-02, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 1e+04]
Presolve removed 5 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.6732673e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective -2.673267327e+03
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 6 columns and 13 nonzeros
Coef

In [125]:
m.write("part3.lp")

In [126]:
results_df = pd.DataFrame(results)
results_df.columns = ["s0", "s1", "s2", "s3", "s4", "权重", "风险"]
results_df

,s0,s1,s2,s3,s4,权重,风险
0,0.0,9900.990099,0.000000,0.000000,0.000000,0.00,247.524752
1,0.0,9900.990099,0.000000,0.000000,0.000000,0.01,247.524752
2,0.0,9900.990099,0.000000,0.000000,0.000000,0.02,247.524752
3,0.0,9900.990099,0.000000,0.000000,0.000000,0.03,247.524752
4,0.0,9900.990099,0.000000,0.000000,0.000000,0.04,247.524752
...,...,...,...,...,...,...,...
91,0.0,2375.839539,3959.732566,1079.927063,2284.461096,0.91,59.395988
92,0.0,2375.839539,3959.732566,1079.927063,2284.461096,0.92,59.395988
93,0.0,2375.839539,3959.732566,1079.927063,2284.461096,0.93,59.395988
94,0.0,2375.839539,3959.732566,1079.927063,2284.461096,0.94,59.395988
